This Notebook accompanies the submission of the paper "Causal Discovery in Time Series Data Using Causally Invariant Locally Linear Models" submitted to the NeuRIPS 2022 Workshop "A Causal View on Dynamical Systems". Running this Notebook in order will create all figures of the main text.

In [ ]:
### In this Notebook we will experiment with the effect of small local linear intervals for non-linear data
### Some initializations
import numpy as np
import math
import random
import copy
import matplotlib.pyplot as plt
import scipy.stats
import warnings
import itertools

In [ ]:
### Here we use i.i.d data to create Figure 1a
np.random.seed(0)
d=5  #Dimensionality
s=2  #Number of support entries
samples=[3,4,5,6,7,8,9,10]  #Sample Sizes
success=np.zeros(len(samples))
runs=100
I=100      #Number of Intervals 
lam=0.01   #Lambda
a=[list(itertools.combinations((0,1,2,3,4), k)) for k in range(0,d)]
subsets = [item for sublist in a for item in sublist]
dic={}
for o,n in enumerate(samples):
    print('Sample Size',o+1,'of',len(samples))
    for r in range(runs):
        x=np.zeros((I,n,d))
        y=np.zeros((I,n))
        betas=np.zeros((I,d))
        R=np.zeros((I,n))
        for i in range(I):
            b=np.random.uniform(low=0,high=30,size=(d))
            a=np.random.uniform(low=-30,high=0,size=(d))
            for p in range(d):
                x[i,:,p]=np.random.uniform(low=a[p],high=b[p],size=(n))
            betas[i,:s]=np.random.uniform(low=-1,high=4,size=(s))
            e=np.random.multivariate_normal(np.zeros(n),np.eye(n))
            y[i,:]=x[i,:,:]@betas[i,:]+e

        for ind in subsets:
            for i in range(I):
                if len(ind)==0:
                    R[i,:]=y[i,:]
                else:
                    ind=np.array(ind)
                    beta_hat=np.linalg.inv(x[i,:,ind]@x[i,:,ind].T+lam*np.eye(len(ind)))@(x[i,:,ind])@y[i,:]
                    R[i,:]=y[i,:]-x[i,:,ind].T@beta_hat
            dic[tuple(ind)]=copy.copy(R)
        S_ini=[]
        for k in dic:
            Res=dic[k]
            pval=scipy.stats.anderson_ksamp(Res)[2]
            if pval>0.05:
                S_ini.append(k)
        plausibleS=[set(S_ini[k]) for k in range(len(S_ini))]
        if not not plausibleS:
            if tuple(set.intersection(*plausibleS))==(0,1):
                success[o]+=1
success=success/runs
fig, ax = plt.subplots()
for label in (ax.get_xticklabels() + ax.get_yticklabels()):
	label.set_fontsize(14)
plt.plot(samples,success,linewidth=2.5)
plt.xlabel('Sample Size',fontsize=14)
plt.ylabel('Success Rate',fontsize=14)
plt.savefig('andersIID.eps', format='eps')

In [ ]:
### Here we use i.i.d data to create Figure 1b
np.random.seed(0)
d=5  #Dimensionality
s=2  #Number of support entries
n=4  #Sample size
lambdas=[0.001,0.005,0.01,0.05,0.1,0.15]
success2=np.zeros(len(lambdas))
kappa=1
runs=100
I=100
a=[list(itertools.combinations((0,1,2,3,4), k)) for k in range(0,d)]
subsets = [item for sublist in a for item in sublist]
dic={}
for o,lam in enumerate(lambdas):
    print('Lambda number',o+1,'of',len(lambdas))
    for r in range(runs):
        x=np.zeros((I,n,d))
        y=np.zeros((I,n))
        betas=np.zeros((I,d))
        R=np.zeros((I,n))
        for i in range(I):
            b=np.random.uniform(low=0,high=30,size=(d))
            a=np.random.uniform(low=-30,high=0,size=(d))
            for p in range(d):
                x[i,:,p]=np.random.uniform(low=a[p],high=b[p],size=(n))
            betas[i,:s]=np.random.uniform(low=-1,high=4,size=(s))
            e=np.random.multivariate_normal(np.zeros(n),np.eye(n))
            y[i,:]=x[i,:,:]@betas[i,:]+e

        for ind in subsets:
            for i in range(I):
                if len(ind)==0:
                    R[i,:]=y[i,:]
                else:
                    ind=np.array(ind)
                    beta_hat=np.linalg.inv(x[i,:,ind]@x[i,:,ind].T+lam*np.eye(len(ind)))@(x[i,:,ind])@y[i,:]
                    R[i,:]=y[i,:]-x[i,:,ind].T@beta_hat
            dic[tuple(ind)]=copy.copy(R)
        S_ini=[]
        for k in dic:
            Res=dic[k]
            if np.var(np.var(Res,axis=1))<1:
                S_ini.append(k)
        plausibleS=[set(S_ini[k]) for k in range(len(S_ini))]
        if not not plausibleS:
            if tuple(set.intersection(*plausibleS))==(0,1):
                success2[o]+=1
success2=success2/runs
fig, ax = plt.subplots()
for label in (ax.get_xticklabels() + ax.get_yticklabels()):
	label.set_fontsize(14)
plt.plot(lambdas,success2,linewidth=2.5)
plt.xlabel('Lambda',fontsize=14)
plt.ylabel('Success Rate',fontsize=14)
plt.savefig('variationIID.eps', format='eps')

In [ ]:
### Here we use vAR data to create Figure 2a

Intervals=[20,25,33,50,100,200,400,800]  #Total modeled intervals, the modelled sample size will follow
runs=100
success3=np.zeros(len(Intervals))
sample_size=np.zeros(len(Intervals))
n_switch=40  # Ground truth sample size per interval
np.random.seed(0)
d=5 #Dimensionality
target=1 #Target covariate
length=4000  #Total length of the time series
for p,I in enumerate(Intervals):
    print('Interval',p+1,'of',len(Intervals))
    for r in range(runs):
        n=length//I  #Model sample size, equivalent to hat(n) in the paper
        sample_size[p]=n
        series=np.zeros((length,d))
        x=np.zeros((I,n,d))
        y=np.zeros((I,n))
        A=np.array([[0.5,-0.7,0,0,0.9],[0,0.4,1,0,0.5],[0,0,-0.9,0,0],[0,0.5,0,0.8,-1],[0,0,0,0,0.4]])
        supp=copy.copy(A)
        supp[supp!=0]=1
        non_supp=np.argwhere(A == 0)
        mean=np.zeros((d))
        cov=np.eye(d)
        series[0]=np.random.multivariate_normal(mean,4*np.eye(d))
        count=1
        i=-1
        b=np.random.uniform(low=0,high=30)
        a=np.random.uniform(low=-30,high=0)
        for j in range(0,n*I-1):
            series[j+1] =np.matmul(series[j],A)
            series[j+1,np.delete(np.array([0,1,2,3,4]),target)] =series[j+1,np.delete(np.array([0,1,2,3,4]),target)]+ np.random.uniform(low=a,high=b,size=(d-1))
            series[j+1,target] =series[j+1,target]+ np.random.uniform(low=-1,high=1)
            A=A+np.random.normal(loc=-0.0001,scale=0.001,size=(5,5))*supp
            A[A>0.9]=0.8
            A[A<-0.9]=-0.8
            if j%n_switch==0:
                b=np.random.uniform(low=0,high=30)
                a=np.random.uniform(low=-30,high=0)
            if j%n==0:
                i+=1
            x[i,j-i*n,:]=series[j,:]
            y[i,j-i*n]=series[j+1,target]
            y[i,j-i*n]=series[j+1,target]+np.random.normal(loc=0,scale=1)

        ## Here we perform the hypothesis test
        R=np.zeros((I,n))
        a=[list(itertools.combinations((0,1,2,3,4), k)) for k in range(0,d)]
        subsets = [item for sublist in a for item in sublist]
        dic={}

        for ind in subsets:
            for i in range(I):
                if len(ind)==0:
                    R[i,:]=y[i,:]
                else:
                    ind=np.array(ind)
                    beta_hat=np.linalg.inv(x[i,:,ind]@x[i,:,ind].T+0.01*np.eye(len(ind)))@(x[i,:,ind])@y[i,:]
                    R[i,:]=y[i,:]-x[i,:,ind].T@beta_hat


            dic[tuple(ind)]=copy.copy(R)
        S_ini=[]
        for k in dic:
            Res=dic[k]
            pval=scipy.stats.anderson_ksamp(Res)[2]
            if pval>0.05:
                S_ini.append(k)
        plausibleS=[set(S_ini[k]) for k in range(len(S_ini))]
        if not not plausibleS:
            if tuple(set.intersection(*plausibleS))==tuple((np.argwhere(A[:,target]!=0).flatten())):
                success3[p]+=1

success3=success3/runs
fig, ax = plt.subplots()
for label in (ax.get_xticklabels() + ax.get_yticklabels()):
	label.set_fontsize(14)
plt.plot(sample_size,success3,linewidth=2.5)
plt.xlabel('Sample Size',fontsize=14)
plt.ylabel('Success Rate',fontsize=14)
plt.savefig('andersonVAR.eps', format='eps')

In [ ]:
### Here we use VAR data to create Figure 2b

## Here we generate the data
I=1000  #Number of intervals, which leaves a model sample size (hat(n)) of 4
lambdas=[0.01,0.05,0.1,0.15,0.2,0.25]
sample_size=np.zeros(len(lambdas))
runs=100
success4=np.zeros(len(lambdas))
n_switch=40  #Ground truth sample size
np.random.seed(0)
d=5 #Dimensionality
target=1 #Target covariate
length=4000  #Total length of the time series
for p,lam in enumerate(lambdas):
    print('Lambda',p+1,'of',len(lambdas))
    for r in range(runs):
        n=length//I  #Model sample size (hat(n))
        sample_size[p]=n
        series=np.zeros((length,d))
        x=np.zeros((I,n,d))
        y=np.zeros((I,n))
        A=np.array([[0.5,-0.7,0,0,0.9],[0,0.4,1,0,0.5],[0,0,-0.9,0,0],[0,0.5,0,0.8,-1],[0,0,0,0,0.4]])
        supp=copy.copy(A)
        supp[supp!=0]=1
        non_supp=np.argwhere(A == 0)
        mean=np.zeros((d))
        cov=np.eye(d)
        series[0]=np.random.multivariate_normal(mean,4*np.eye(d))
        count=1
        i=-1
        b=np.random.uniform(low=0,high=30)
        a=np.random.uniform(low=-30,high=0)
        for j in range(0,n*I-1):
            series[j+1] =np.matmul(series[j],A)
            series[j+1,np.delete(np.array([0,1,2,3,4]),target)] =series[j+1,np.delete(np.array([0,1,2,3,4]),target)]+ np.random.uniform(low=a,high=b,size=(d-1))
            A=A+np.random.normal(loc=-0.001,scale=0.001,size=(5,5))*supp
            A[A>0.9]=0.8
            A[A<-0.9]=-0.8
            if j%n_switch==0:
                b=np.random.uniform(low=0,high=30)
                a=np.random.uniform(low=-30,high=0)
            if j%n==0:
                i+=1
            x[i,j-i*n,:]=series[j,:]
            y[i,j-i*n]=series[j+1,target]
            y[i,j-i*n]=series[j+1,target]+np.random.normal(loc=0,scale=1)

        ## Here we perform the variance test
        R=np.zeros((I,n))
        a=[list(itertools.combinations((0,1,2,3,4), k)) for k in range(0,d)]
        subsets = [item for sublist in a for item in sublist]
        dic={}

        for ind in subsets:
            for i in range(I):
                if len(ind)==0:
                    R[i,:]=y[i,:]
                else:
                    ind=np.array(ind)
                    beta_hat=np.linalg.inv(x[i,:,ind]@x[i,:,ind].T+lam*np.eye(len(ind)))@(x[i,:,ind])@y[i,:]
                    R[i,:]=y[i,:]-x[i,:,ind].T@beta_hat


            dic[tuple(ind)]=copy.copy(R)
        S_ini=[]
        for k in dic:
            Res=dic[k]
            if np.var(np.var(Res,axis=1))<1:
                S_ini.append(k)
        plausibleS=[set(S_ini[k]) for k in range(len(S_ini))]
        if not not plausibleS:
            if tuple(set.intersection(*plausibleS))==tuple((np.argwhere(A[:,target]!=0).flatten())):
                success4[p]+=1
success4=success4/runs
fig, ax = plt.subplots()
for label in (ax.get_xticklabels() + ax.get_yticklabels()):
	label.set_fontsize(14)
plt.plot(lambdas,success4,linewidth=2.5)
plt.xlabel('Lambda',fontsize=14)
plt.ylabel('Success Rate',fontsize=14)
plt.savefig('variationVAR.eps', format='eps')